In [ ]:
# -*- coding: utf-8 -*-
# ---
# jupyter:
#   jupytext:
#     text_representation:
#       extension: .py
#       format_name: light
#       format_version: '1.5'
#       jupytext_version: 1.14.0
#   kernelspec:
#     display_name: Python 3 (ipykernel)
#     language: python
#     name: python3
# ---

# # 3. Thử nghiệm Khôi phục Ảnh (Loại bỏ nhiễu)

# Mục tiêu: Áp dụng các bộ lọc không gian để giảm nhiễu.

# ## Import thư viện
import os
import sys
import cv2
import numpy as np

# Thêm thư mục src vào sys.path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import utils
from src import restoration


In [ ]:
# ## Hàm tạo nhiễu (để thử nghiệm)
def add_salt_pepper_noise(image, salt_prob=0.01, pepper_prob=0.01):
    """Thêm nhiễu muối tiêu vào ảnh."""
    noisy_image = np.copy(image)
    total_pixels = image.size
    
    # Thêm muối (pixel trắng)
    num_salt = np.ceil(salt_prob * total_pixels)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = (255, 255, 255) if len(image.shape) > 2 else 255
    
    # Thêm tiêu (pixel đen)
    num_pepper = np.ceil(pepper_prob * total_pixels)
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape[:2]]
    noisy_image[coords[0], coords[1]] = (0, 0, 0) if len(image.shape) > 2 else 0
    
    return noisy_image


In [ ]:
def add_gaussian_noise(image, mean=0, sigma=25):
    """Thêm nhiễu Gaussian vào ảnh."""
    row, col, ch = image.shape if len(image.shape) > 2 else (image.shape[0], image.shape[1], 1)
    gauss = np.random.normal(mean, sigma, (row, col, ch))
    gauss = gauss.reshape(row, col, ch) if ch > 1 else gauss.reshape(row, col)
    noisy_image = image + gauss
    # Đảm bảo giá trị pixel nằm trong khoảng 0-255
    noisy_image = np.clip(noisy_image, 0, 255) 
    return noisy_image.astype(np.uint8)


In [ ]:
# ## Tải ảnh mẫu
image_filename = 'sample_sign.jpg' # Đảm bảo ảnh này tồn tại
image_path = os.path.join('..', 'data', 'raw', image_filename) 
img_original = utils.load_image(image_path, cv2.IMREAD_COLOR)

if img_original is not None:
    # ## Tạo ảnh nhiễu để thử nghiệm
    img_noisy_sp = add_salt_pepper_noise(img_original, salt_prob=0.02, pepper_prob=0.02)
    img_noisy_gaussian = add_gaussian_noise(img_original, sigma=30)

    # ## Áp dụng các bộ lọc khôi phục

    # 1. Lọc nhiễu muối tiêu bằng Median Filter
    img_restored_median = restoration.apply_median_filter(img_noisy_sp, kernel_size=3)

    # 2. Lọc nhiễu Gaussian bằng Mean Filter
    img_restored_mean = restoration.apply_mean_filter(img_noisy_gaussian, kernel_size=(3, 3))

    # 3. Lọc nhiễu Gaussian bằng Gaussian Blur (thường cho kết quả tốt hơn Mean)
    img_restored_gaussian = restoration.apply_gaussian_blur(img_noisy_gaussian, kernel_size=(5, 5))

    # ## Hiển thị kết quả

    # So sánh lọc nhiễu muối tiêu
    utils.display_images(
        ["Ảnh Gốc", "Nhiễu Muối Tiêu", "Lọc Median"],
        [img_original, img_noisy_sp, img_restored_median],
        figsize=(18, 6)
    )

    # So sánh lọc nhiễu Gaussian
    utils.display_images(
        ["Ảnh Gốc", "Nhiễu Gaussian", "Lọc Mean", "Lọc Gaussian Blur"],
        [img_original, img_noisy_gaussian, img_restored_mean, img_restored_gaussian],
        figsize=(24, 6)
    )
else:
    print(f"Lỗi: Không thể tải ảnh tại '{image_path}'.")